In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import numpy as np

data_dir = 'D:/tester01'

image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    for image_file in os.listdir(label_path):
        image_paths.append(os.path.join(label_path, image_file))
        labels.append(label)

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

images = [np.array(Image.open(img_path).resize((224, 224))) / 255.0 for img_path in image_paths]

X = np.array(images, dtype=np.float32)
y = np.array(labels_encoded)

X_train_temp, X_temp, y_train_temp, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 使用预训练的 MobileNet
mobilenet_model = tf.keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

mobilenet_model.trainable = False

model = Sequential([
    mobilenet_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_temp, y_train_temp, epochs=5, batch_size=32, validation_data=(X_val, y_val))

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Epoch 1/5
39/39 [==============================] - 18s 400ms/step - loss: 3.9425 - accuracy: 0.7951 - val_loss: 0.4785 - val_accuracy: 0.9358
Epoch 2/5
39/39 [==============================] - 15s 378ms/step - loss: 0.0903 - accuracy: 0.9812 - val_loss: 0.2934 - val_accuracy: 0.9532
Epoch 3/5
39/39 [==============================] - 15s 377ms/step - loss: 0.0134 - accuracy: 0.9967 - val_loss: 0.1949 - val_accuracy: 0.9674
Epoch 4/5
39/39 [==============================] - 15s 395ms/step - loss: 0.0014 - accuracy: 0.9992 - val_loss: 0.2116 - val_accuracy: 0.9717
Epoch 5/5
29/29 [==============================] - 6s 204ms/step - loss: 0.1983 - accuracy: 0.9609
Test Loss: 0.19831275939941406, Test Accuracy: 0.960869550704956


In [ ]:
model.save('bl.h5')